Steps to mount drive

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Factify_Synth
# change the location to that folder which contains and the dataset and this code file

/content/drive/MyDrive/Factify_Synth


Installing necessary modules and libraries

In [ ]:
%pip install --quiet --upgrade diffusers transformers accelerate mediapy triton scipy ftfy spacy==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 11.9 MB/s eta 0:00:00


In [ ]:
# The xformers package is mandatory to be able to create several 768x768 images.
%pip install -q xformers==0.0.16rc425

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 79.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 

Importing necessary libraries

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, DPMSolverMultistepScheduler

In [ ]:
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline
import random
import pandas as pd
import numpy as np

Decide the model

In [ ]:
model_id = "stabilityai/stable-diffusion-2-1-base"
# model_id = "stabilityai/stable-diffusion-2-1"
# model_id = "dreamlike-art/dreamlike-photoreal-2.0"

Decide which scheduler will be needed

In [ ]:
scheduler = None
# scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")

Decide the device

In [ ]:
device = "cuda"

Model Parameters

In [ ]:
if model_id.startswith("stabilityai/"):
  model_revision = "fp16"
else:
  model_revision = None

Create model pipeline

In [ ]:
if scheduler is None:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      revision=model_revision,
      )
else:
  pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      scheduler=scheduler,
      torch_dtype=torch.float16,
      revision=model_revision,
      )

seed = random.randint(0, 2147483647)

vae/diffusion_pytorch_model.safetensors not found


Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Shift pipeline to intended device

In [ ]:
pipe = pipe.to(device)
pipe.enable_xformers_memory_efficient_attention()

Generated Image Size Parameters

In [ ]:
if model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768

Sample prompt

In [ ]:
prompt = "a photo of Pikachu fine dining with a view to the Eiffel Tower"

Decide the number of generated image samples you need for each prompt

In [ ]:
num_images = 8

Decide whether negative prompts need to be omitted or not

In [ ]:
remove_safety = False # can be changed to True when and if needed
if remove_safety:
  negative_prompt = None
  pipe.safety_checker = None
else:
  negative_prompt = "nude, naked"

Image Creation, Display and Storing

In [ ]:
images = pipe(
    prompt,
    height = image_length,
    width = image_length,
    num_inference_steps = 500,
    guidance_scale = 9,
    num_images_per_prompt = num_images,
    negative_prompt = negative_prompt,
    generator = torch.Generator("cuda").manual_seed(seed)
    ).images

media.show_images(images)
display(f"Seed: {seed}") # displaying all images created
images[0].save("output.jpg") # saving best image. complete path to save location can be provided here.

Readind the dataset

In [ ]:
df = pd.read_excel("all_claims.xlsx")

In [ ]:
df.head()

,claim,tag
0,Skagen Painter Peder Severin Krøyer favored na...,hover
1,Skagen Painter Peder Severin Krøyer favored na...,hover
2,Skagen Painter Peder Severin Krøyer favored na...,hover
3,"Skagen Painter, who painted the 1893 painting ...",hover
4,Skagen Painter Peder Severin Krøyer favored na...,hover


In [ ]:
df.shape

(352955, 2)

Trying image generation for 5 claims

In [ ]:
for i in range(5):
  claim_text = df.claim[i]
  images = pipe(
    claim_text,
    height = image_length,
    width = image_length,
    num_inference_steps = 25, # this is just for trial. change to >=500 for good results
    guidance_scale = 9,
    num_images_per_prompt = num_images,
    negative_prompt = negative_prompt,
    generator = torch.Generator("cuda").manual_seed(seed)
    ).images
  print("Claim:",claim_text)
  media.show_images(images)
  display(f"Seed: {seed}") # displaying all images created
  # add code for saving image here with location
  print()
  print("--------------------------------------------------------------------------")
  print()